# Day 3: Spiral Memory

You come across an experimental new kind of memory stored on an infinite two-dimensional grid.

Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```

While this is very space-efficient (no squares are skipped), requested data must be carried back to square 1 (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the Manhattan Distance between the location of the data and square 1.

In [1]:
from math import sqrt, ceil


def radius(location):
    """return radius of the part of the spiral that contains the given location."""
    return ceil((sqrt(location) - 1) / 2)


assert radius(1) == 0
assert all(radius(loc) == 1 for loc in range(2, 9 + 1))
assert all(radius(loc) == 2 for loc in range(10, 25 + 1))
assert radius(26) == 3


def midpoints(radius):
    """return indices of midpoints of the spiral's edges at given radius."""
    if radius == 0: return [1]
    east = (2 * radius - 1)**2 + radius
    return [east + k * radius for k in [0, 2, 4, 6]]


assert midpoints(0) == [1]
assert midpoints(1) == [2, 4, 6, 8]
assert midpoints(2) == [11, 15, 19, 23]


def manhattan(index):
    r = radius(index)
    delta = min(abs(index - m) for m in midpoints(r))
    return r + delta

For example:

In [2]:
# data from square 1 is carried 0 steps, since it's at the access port
assert manhattan(1) == 0

# data from square 12 is carried 3 steps, such as: down, left, left
assert manhattan(12) == 3

# data from square 23 is carried only 2 steps: up twice
assert manhattan(23) == 2

# data from square 1024 must be carried 31 steps
assert manhattan(1024) == 31

How many steps are required to carry the data from the square identified in your puzzle input all the way to the access port?

In [3]:
puzzle = 368078
manhattan(puzzle)

371

## Part Two

As a stress test on the system, the programs here clear the grid and then store the value 1 in square 1. Then, in the same allocation order as shown above, they store the sum of the values in all adjacent squares, including diagonals.

In [4]:
def spiral():
    """Generate Cartesian coordinates of (infinite) spiral."""
    pos, ring, state = [0, 0], 0, 'RIGHT'

    while True:
        yield tuple(pos)
        if state == 'UP':
            pos[1] += 1
            if pos[1] == ring:
                state = 'LEFT'
        elif state == 'LEFT':
            pos[0] -= 1
            if pos[0] == -ring:
                state = 'DOWN'
        elif state == 'DOWN':
            pos[1] -= 1
            if pos[1] == -ring:
                state = 'RIGHT'
        elif state == 'RIGHT':
            pos[0] += 1
            if pos[0] == ring + 1:
                state = 'UP'
                ring += 1


from itertools import islice
first_11 = list(islice(spiral(), 11))
assert first_11 == [(0, 0), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1),
                    (0, -1), (1, -1), (2, -1), (2, 0)]

In [5]:
import numpy as np


def iterate_sums_of_neighbors(max_radius, callback=None):
    # pad with empty ring
    k = max_radius + 1
    padded_grid = np.zeros(shape=(2 * k + 1, 2 * k + 1), dtype=int)

    # store value 1 at location 1
    padded_grid[k, k] = 1

    # iteratively compute sums of neighbors
    sp = spiral()
    next(sp)
    for x, y in sp:
        if max(x, y) > max_radius:
            break

        # compute sum of neighbors
        s = 0
        for dx in -1, 0, 1:
            for dy in -1, 0, 1:
                if dx != 0 or dy != 0:
                    s += padded_grid[k - (y + dy), k + (x + dx)]
        padded_grid[k - y, k + x] = s

        # invoke callback?
        if callback is not None:
            result = callback(s)
            if result is not None:
                return result

    return padded_grid[1:-1, 1:-1]

So, the first few squares' values are chosen as follows:

- Square 1 starts with the value 1.
- Square 2 has only one adjacent filled square (with value 1), so it also stores 1.
- Square 3 has both of the above squares as neighbors and stores the sum of their values, 2.
- Square 4 has all three of the aforementioned squares as neighbors and stores the sum of their values, 4.
- Square 5 only has the first and fourth squares as neighbors, so it gets the value 5.

Once a square is written, its value does not change. Therefore, the first few squares would receive the following values:

In [6]:
assert np.allclose(
    iterate_sums_of_neighbors(max_radius=2), [
        [147, 142, 133, 122, 59],
        [304, 5, 4, 2, 57],
        [330, 10, 1, 1, 54],
        [351, 11, 23, 25, 26],
        [362, 747, 806, 880, 931],
    ])

What is the first value written that is larger than your puzzle input?

In [7]:
def first_value_above_threshold(max_radius, threshold):
    def callback(sum):
        if sum > threshold:
            return sum

    return iterate_sums_of_neighbors(max_radius, callback)


assert first_value_above_threshold(2, 53) == 54
assert first_value_above_threshold(2, 748) == 806

Real puzzle:

In [8]:
first_value_above_threshold(radius(puzzle), puzzle)

369601